### Unidad 2: Taller de análisis de datos textuales

<h1> Notebook 3 - Introducción a la vectorización de palabras: Word2Vec</h1>

En los notebooks anteriores, hemos hablado de los vectores que se utilizan para representar nuestros datos textuales en una forma matemática, y así poder aplicar métodos de machine learning.

En este notebook, vamos a llevar esta idea un paso más allá y generar reprentaciones vectoriales de las <u>palabras</u>. Esta idea es generalmente llamada __Word Embeddings__. __Word2Vec__ es un algoritmo popular para implementar esta idea.
Esta técnica permite para representar mejor el significado de una palabra.

Para más detalles sobre los fundamentos de Word2Vec, se puede leer: _Efficient Estimation of Word Representations in Vector Space_ [[Mikolov et al., 2013]](https://arxiv.org/pdf/1301.3781.pdf)

## 1. ¿Qué es una representación vectorial de una palabra?

Supongamos que nuestro vocabulario tiene sólo cinco palabras: King, Queen, Man, Woman y Child. Podríamos codificar la palabra 'Queen' como:

<img src="word2vec1.png"/>

Usando tal codificación, no hay comparación interesante que podamos hacer entre vectores de palabras. Con word2vec, se utiliza una representación distribuida de una palabra. Cada palabra está representada por una distribución de pesos entre esos elementos. La representación de una palabra se extiende a través de todos los elementos en el vector, y cada elemento en el vector contribuye a la definición de muchas palabras.

<img src="word2vec2.png"/>

Tal vector llega a representar de alguna manera abstracta el 'significado' de una palabra. Y como veremos a continuación, simplemente examinando un corpus grande es posible aprender vectores de palabras que son capaces de capturar las relaciones entre palabras de una manera sorprendentemente expresiva. 


### Razonamiento con vectores de palabras

Los vectores son muy buenos para responder a problemas de analogía de tipo: '_A_ es a _B_ lo que _C_ es a ...?'. Por ejemplo, el _hombre_ es a la _mujer_ lo que el _tío_ es a la _tía_ usando un método de desplazamiento vectorial.

Por ejemplo, el desplazamiento vectorial que ilustra la _relación de género_:

<img src="word2vec3.png"/>


Este tipo de composición vectorial también nos permite responder a la pregunta __King - Man + Woman = ?__ y llegar al resultado __Queen__.

<img src="word2vec4.png"/>

Estos modelos vectoriales de palabras están destacables ya que permiten representar ciertas relaciones entre palabras sin aportar información sobre la semántica de las palabras. Como vamos a verlo, se pueden aprender los vectores a partir de gran conjuntos de datos textuales.

### Construir una representación vectorial de una palabra _y_ considerando las palabras X que aparacen frecuentemente en su contexto

Para construir sus vectores, Word2Vec utiliza un dataset de entrenamiento y algoritmos de aprendizaje basados en redes neuronales (__Continuous Bag of Words__ (CBOW), o modelo __Skip Gram__). El objetivo de esta fase de aprendizaje es aprender cuáles son las palabras _X_ más probables de aparecer en el contexto de una palabra _y_.

<img src="word2vec5.png"/>

Por ejemplo, ¿cuál es la probabilidad de tener la palabra 'perro' dada la palabra 'pelota', considerando una ventana de tamaño 3:

<code>Los expertos explican que los __perros__ persiguen __pelotas__ en movimiento como parte de un comportamiento instintivo. Aunque no todos los perros tienen tan despiertos su instinto de caza, esto no impide que la mayoría de ellos sí disfruten, y mucho, de los juegos que incluyen persecuciones de una saltarina __pelota__ que bota delante de ellos. </code>

## 2. Aplicar Word2Vec con Gensim para extraer representaciones vectoriales de palabras en inglés

La clase <code>word2vec</code> de Gensim permite entrenar modelos vectoriales de palabras (ver documentación: https://radimrehurek.com/gensim/models/word2vec.html).

Esta clase tiene varios parametros, en particular:
- <code>sentences</code>: una lista de palabras o de frases que sirve para entrenar el modelo
- <code>sg</code>: define que algoritmos de aprendizaje utilizar (0=CBOW, 1=skip-gram)
- <code>size</code>: define la dimensión de los vectores que se desea extraer
- <code>window</code>: define el número de palabras considerar a la izquierda y a la derecha de una palabra
- <code>min_count</code>: ignorar las palabras que aparecen menos de _min_count_
y otros asociados a la parametrización de la fase de aprendizaje de la red neuronal (que no detallaremos en esta parte del curso):
- <code>alpha</code>: el _learning rate_ utilizado para optimizar los parametros de la red neuronal.
- <code>iter</code>: número de iteraciones (epocas) sobre el dataset para encontrar los parametreos que optimizan la red neuronal.

In [41]:
from gensim.models import word2vec

Para entrenar nuestro modelo Word2Vec, podemos utilizar nuestros propios datasets o utilizar datasets genericos existentes. Para empezar, utilizaremos 100 MB de textos extraidos de Wikipedia en inglés, para generar vectores de 200 dimensiones.

In [71]:
sentences = word2vec.Text8Corpus('text8.txt')

In [73]:
model = word2vec.Word2Vec(sentences,size=200,hs=1)

In [74]:
print(model)

Word2Vec(vocab=71290, size=200, alpha=0.025)


Ahora que hemos aprendido nuestro modelo, tratemos de resolver la ecuación <code>King - Man + Woman</code>.

En otras palabras buscamos cuál es el vector más similar al vector que adiciona positivamente 'King' y 'Woman' y negativamente 'Man'.

In [75]:
model.wv.most_similar(positive=['woman','king'],negative=['man'],topn=1)[0]

('queen', 0.5662363767623901)

In [76]:
model.wv.most_similar_cosmul(positive=['woman','king'],negative=['man'])

[('queen', 0.883696973323822),
 ('empress', 0.8332663178443909),
 ('throne', 0.8241393566131592),
 ('matilda', 0.8198255896568298),
 ('jadwiga', 0.8192695379257202),
 ('pepin', 0.8093411326408386),
 ('princess', 0.8090969920158386),
 ('daughter', 0.8072876930236816),
 ('monarch', 0.8046062588691711),
 ('isabella', 0.7938171625137329)]

In [124]:
model.wv.most_similar(positive=["fascism","peace","justice"])

[('unity', 0.5120744109153748),
 ('reconciliation', 0.5073227882385254),
 ('communism', 0.48326578736305237),
 ('bolshevism', 0.4750584065914154),
 ('fascist', 0.4749867916107178),
 ('socialism', 0.4738006889820099),
 ('democracy', 0.4734036922454834),
 ('conciliation', 0.461652010679245),
 ('neutrality', 0.45902019739151),
 ('treaties', 0.45625239610671997)]

In [32]:
model.wv['computer']
model.save("text8_model")

In [33]:
model.save("text8_model")
model=word2vec.Word2Vec.load("text8_model")

In [34]:
model.wv.doesnt_match("breakfast cereal dinner lunch".split())

/home/mvernier/.local/lib/python3.5/site-packages/gensim/models/keyedvectors.py:876: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'cereal'

In [35]:
model.wv.similarity('woman','man')

0.7032394

In [36]:
model.wv.similarity('woman','cereal')

0.02177691

In [37]:
model.wv.distance('man','woman')

0.296760618686676

## Doc2Vec

In [49]:
import gensim
import os
import collections
import smart_open
import random

In [50]:
gensim.models.doc2vec.LabeledSentence

<function gensim.models.doc2vec.LabeledSentence(words, tags)>

In [51]:
test_data_dir ='{}'.format(os.sep).join([gensim.__path__[0],'test',	'test_data'])
lee_train_file=test_data_dir+os.sep+'lee_background.cor'
lee_test_file=test_data_dir+os.sep+'lee.cor'

In [52]:
def read_corpus(file_name,tokens_only=False):
    with smart_open.smart_open(file_name,encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                yield gensim.utils.simple_preprocess(line)
            else:
                # For training data, add tags
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(line),[i])

In [53]:
train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file,tokens_only=True))

In [54]:
train_corpus[:2]

[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

In [55]:
test_corpus[:2]

[['the',
  'national',
  'executive',
  'of',
  'the',
  'strife',
  'torn',
  'democrats',
  'last',
  'night',
  'appointed',
  'little',
  'known',
  'west',
  'australian',
  'senator',
  'brian',
  'greig',
  'as',
  'interim',
  'leader',
  'shock',
  'move',
  'likely',
  'to',
  'provoke',
  'further',
  'conflict',
  'between',
  'the',
  'party',
  'senators',
  'and',
  'its',
  'organisation',
  'in',
  'move',
  'to',
  'reassert',
  'control',
  'over',
  'the',
  'party',
  'seven',
  'senators',
  'the',
  'national',
  'executive',
  'last',
  'night',
  'rejected',
  'aden',
  'ridgeway',
  'bid',
  'to',
  'become',
  'interim',
  'leader',
  'in',
  'favour',
  'of',
  'senator',
  'greig',
  'supporter',
  'of',
  'deposed',
  'leader',
  'natasha',
  'stott',
  'despoja',
  'and',
  'an',
  'outspoken',
  'gay',
  'rights',
  'activist'],
 ['cash',
  'strapped',
  'financial',
  'services',
  'group',
  'amp',
  'has',
  'shelved',
  'million',
  'plan',
  'to',
 

In [56]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)

/home/mvernier/.local/lib/python3.5/site-packages/gensim/models/doc2vec.py:571: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/mvernier/.local/lib/python3.5/site-packages/gensim/models/doc2vec.py:575: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")


In [57]:
model.build_vocab(train_corpus)

In [58]:
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """Entry point for launching an IPython kernel.


In [59]:
model.infer_vector(['only', 'you', 'can', 'prevent', 'forrest', 'fires'])

array([ 0.28578985,  0.24217391, -0.20216593, -0.05530727, -0.18294808,
       -0.3028205 ,  0.43821105,  0.19100691, -0.32210562,  0.14483336,
       -0.15796368, -0.10855799,  0.35547626,  0.08085938, -0.1515045 ,
        0.15593642,  0.09760015, -0.04266372,  0.12743095, -0.07318401,
       -0.07034647, -0.34250104,  0.14843975,  0.01175584, -0.19038928,
       -0.34336922,  0.23858722, -0.1920204 , -0.06575964, -0.12403026,
       -0.18707556,  0.119217  ,  0.29471603,  0.32173702, -0.03061843,
        0.258997  , -0.00804112,  0.05417662, -0.0850556 , -0.18037279,
        0.01045722,  0.07649703,  0.21431218, -0.1479705 ,  0.29921138,
        0.33175048, -0.05683559,  0.22207293, -0.36630762, -0.19607906],
      dtype=float32)

In [60]:
ranks = []
second_ranks = []
for doc_id in range(len(train_corpus)):
    inferred_vector = model.infer_vector(train_corpus[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)
    
    second_ranks.append(sims[1])

In [61]:
inferred_vector = model.infer_vector(train_corpus[0].words)
sims = model.docvecs.most_similar([inferred_vector])

In [62]:
sims

[(0, 0.9725579619407654),
 (48, 0.8745595812797546),
 (40, 0.8412767648696899),
 (255, 0.8355098366737366),
 (272, 0.7915453910827637),
 (33, 0.7869666814804077),
 (8, 0.7693777680397034),
 (264, 0.7450066804885864),
 (10, 0.6932415962219238),
 (25, 0.684721052646637)]

In [63]:
' '.join(train_corpus[0].words)

'hundreds of people have been forced to vacate their homes in the southern highlands of new south wales as strong winds today pushed huge bushfire towards the town of hill top new blaze near goulburn south west of sydney has forced the closure of the hume highway at about pm aedt marked deterioration in the weather as storm cell moved east across the blue mountains forced authorities to make decision to evacuate people from homes in outlying streets at hill top in the new south wales southern highlands an estimated residents have left their homes for nearby mittagong the new south wales rural fire service says the weather conditions which caused the fire to burn in finger formation have now eased and about fire units in and around hill top are optimistic of defending all properties as more than blazes burn on new year eve in new south wales fire crews have been called to new fire at gunning south of goulburn while few details are available at this stage fire authorities says it has clo

In [64]:
collections.Counter(ranks) 

Counter({0: 292, 1: 8})

In [65]:
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Document (299): «australia will take on france in the doubles rubber of the davis cup tennis final today with the tie levelled at wayne arthurs and todd woodbridge are scheduled to lead australia in the doubles against cedric pioline and fabrice santoro however changes can be made to the line up up to an hour before the match and australian team captain john fitzgerald suggested he might do just that we ll make team appraisal of the whole situation go over the pros and cons and make decision french team captain guy forget says he will not make changes but does not know what to expect from australia todd is the best doubles player in the world right now so expect him to play he said would probably use wayne arthurs but don know what to expect really pat rafter salvaged australia davis cup campaign yesterday with win in the second singles match rafter overcame an arm injury to defeat french number one sebastien grosjean in three sets the australian says he is happy with his form it not v

In [66]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(train_corpus) - 1)

# Compare and print the most/median/least similar documents from the train corpus
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_corpus[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_corpus[sim_id[0]].words)))

Train Document (265): «the federal government is under fire from unions over new departmental report which recommends australia outsource information technology it to india the document says india has low cost skilled workforce the minister for foreign affairs and trade alexander downer has given his support to the document from his department entitled india new economy old economy the report says sectors like it finance and offer attractive direct investment opportunities it also says australian firms could become more competitive by outsourcing to the indian it sector the community and public sector union wendy caird says the government seems to be encouraging local companies to export jobs to india think that quite alarming obviously labour is great deal cheaper in india and that assisted by the indian government removing labour laws and bankruptcy laws ms caird said the union says while the initiative may create jobs in india it will not help australia rising unemployment»

Similar